This notebook studies the same features as the notebook "Effect_mutations_alone.ipynb" but with another method (xgboost)

In [146]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch 
from torch.utils.data import Dataset
import torch.nn as nn
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam

from scipy.stats import spearmanr

from sklearn.metrics import mean_squared_error
from sklearn import metrics

import os
from collections import Counter

In [147]:
import blosum as bl
matrix = bl.BLOSUM(62)

1. Load the dataset 

In [148]:
#load training data (will be put in a function later) 
#TO CHANGE
path = os.getcwd()
for i in range(3) :

    path = os.path.dirname(path)

path += '/data/'
train_df = pd.read_csv(path + 'clean_train_data.csv')
train_df = train_df.drop(columns=['data_source1'])
train_df = train_df.drop(columns=['data_source2'])

train_df = train_df.drop(columns=['protSeq2'])
train_df = train_df.drop(columns=['tm1'])
train_df = train_df.drop(columns=['tm2'])
train_df = train_df.drop(columns=['group1'])
train_df = train_df.drop(columns=['group2'])

train_df = train_df.dropna()

train_df.head()
#dT = 'target'

,protSeq1,operation,position1,position2,change1,change2,pH1,pH2,target
0,MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,replace,2,2,A,D,6.5,6.5,-6.7
1,MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,replace,2,2,A,E,6.5,6.5,-3.9
2,MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,replace,2,2,A,F,6.5,6.5,-1.2
3,MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,replace,2,2,A,G,6.5,6.5,-4.0
4,MNAFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSEL...,replace,2,2,A,L,6.5,6.5,2.7


In [149]:
test_df = pd.read_csv(path+ 'test_mutations.csv', index_col="seq_id")
test_df = test_df.drop(columns=['data_source'])
test_df = test_df.drop(columns=['b_factor'])
test_df = test_df.drop(columns=['bFactorAdj'])
test_df = test_df.drop(columns=['score'])
test_df = test_df.drop(columns=['position1'])
test_df = test_df.drop(columns=['position2'])


test_df.head()

,protein_sequence,pH,modif,score_adj,operation,change1,change2
seq_id,,,,,,,
31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,16,0.880797,replace,E,L
31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,16,0.880797,replace,K,L
31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,16,0.999955,delete,K,NaN
31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,17,0.952574,replace,C,K
31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,17,0.952574,replace,F,K


In [150]:
test_df['length'] = test_df['protein_sequence'].str.len()
test_df = test_df.drop(columns=['protein_sequence'])

In [151]:
train_df = train_df[train_df['operation']=='replace']

In [152]:
#add a new column that contains the length of each protein sequence (before padding)
train_df['length'] = train_df['protSeq1'].str.len()
train_df = train_df.drop(columns=['protSeq1'])



In [153]:
#Put the distance of the 2 aa as a feature, with blosum: 
#hypothesis : this score influences the delta Tm. 
#Allows to encode the "mutation" and the info : which aa into which aa


def blosum_apply(row):
        if (row['operation']=='replace'):
            res = matrix[row['change1'] + row['change2']]
        else:
                res = -10
        return res

train_df['dist_mutation'] = train_df.apply(blosum_apply, axis=1)
test_df['dist_mutation'] = test_df.apply(blosum_apply, axis=1)


    

In [154]:
test_df = test_df.drop(columns=['change1'])
test_df = test_df.drop(columns=['change2'])


In [155]:
train_df = train_df.drop(columns=['change1'])
train_df = train_df.drop(columns=['change2'])

In [156]:
train_df.head()

,operation,position1,position2,pH1,pH2,target,length,dist_mutation
0,replace,2,2,6.5,6.5,-6.7,164,-2.0
1,replace,2,2,6.5,6.5,-3.9,164,-1.0
2,replace,2,2,6.5,6.5,-1.2,164,-2.0
3,replace,2,2,6.5,6.5,-4.0,164,0.0
4,replace,2,2,6.5,6.5,2.7,164,-1.0


In [157]:
test_df.head()

,pH,modif,score_adj,operation,length,dist_mutation
seq_id,,,,,,
31390,8,16,0.880797,replace,221,-3.0
31391,8,16,0.880797,replace,221,-2.0
31392,8,16,0.999955,delete,220,-10.0
31393,8,17,0.952574,replace,221,-3.0
31394,8,17,0.952574,replace,221,-3.0


In [158]:
#Compute the relative position of the mutation

train_df['relative_position'] = train_df['position1']/train_df['length']
train_df = train_df.drop(columns=['position1'])
train_df = train_df.drop(columns=['position2'])





In [159]:
test_df['relative_position'] = test_df['modif']/test_df['length']
test_df = test_df.drop(columns=['modif'])

In [160]:
train_df.head()

,operation,pH1,pH2,target,length,dist_mutation,relative_position
0,replace,6.5,6.5,-6.7,164,-2.0,0.012195
1,replace,6.5,6.5,-3.9,164,-1.0,0.012195
2,replace,6.5,6.5,-1.2,164,-2.0,0.012195
3,replace,6.5,6.5,-4.0,164,0.0,0.012195
4,replace,6.5,6.5,2.7,164,-1.0,0.012195


In [161]:
test_df.head()


,pH,score_adj,operation,length,dist_mutation,relative_position
seq_id,,,,,,
31390,8,0.880797,replace,221,-3.0,0.072398
31391,8,0.880797,replace,221,-2.0,0.072398
31392,8,0.999955,delete,220,-10.0,0.072727
31393,8,0.952574,replace,221,-3.0,0.076923
31394,8,0.952574,replace,221,-3.0,0.076923


In [162]:
#Compute difference of pH
train_df['dPH'] = train_df['pH1']-train_df['pH2']
train_df = train_df.drop(columns=['pH1'])
train_df = train_df.drop(columns=['pH2'])

In [163]:
train_df.head()

,operation,target,length,dist_mutation,relative_position,dPH
0,replace,-6.7,164,-2.0,0.012195,0.0
1,replace,-3.9,164,-1.0,0.012195,0.0
2,replace,-1.2,164,-2.0,0.012195,0.0
3,replace,-4.0,164,0.0,0.012195,0.0
4,replace,2.7,164,-1.0,0.012195,0.0


In [164]:
test_df['dPH'] =test_df['pH'] - 8
test_df = test_df.drop(columns=['pH'])




In [165]:
test_df = test_df.drop(columns=['score_adj'])

In [166]:
test_df = test_df.drop(columns=['operation'])
trian_df = train_df.drop(columns=['operation'])

In [167]:
test_df.head()

,length,dist_mutation,relative_position,dPH
seq_id,,,,
31390,221,-3.0,0.072398,0
31391,221,-2.0,0.072398,0
31392,220,-10.0,0.072727,0
31393,221,-3.0,0.076923,0
31394,221,-3.0,0.076923,0


In [168]:
train_df.head()

,operation,target,length,dist_mutation,relative_position,dPH
0,replace,-6.7,164,-2.0,0.012195,0.0
1,replace,-3.9,164,-1.0,0.012195,0.0
2,replace,-1.2,164,-2.0,0.012195,0.0
3,replace,-4.0,164,0.0,0.012195,0.0
4,replace,2.7,164,-1.0,0.012195,0.0


In [169]:
df = train_df.copy()



In [170]:
df = df.reset_index(drop=True)


In [171]:
test_df = test_df.reset_index(drop=True)

Split to train and validation sets

In [172]:
#split padded_train_df into train and validation sets (will be put in a function later)
train_df , val_df = split_train_test(df,frac=0.8)


In [176]:
#Train the model HGBRegressor
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators = 1000, max_depth = 6)

In [177]:
train_df

,operation,target,length,dist_mutation,relative_position,dPH
0,replace,3.6,164,1.0,0.353659,0.0
1,replace,1.6,537,-1.0,0.197393,0.0
2,replace,-1.3,231,-1.0,0.619048,0.0
3,replace,-5.5,455,-3.0,0.336264,0.0
4,replace,-6.9,231,-1.0,0.619048,0.0
...,...,...,...,...,...,...
1832,replace,1.5,455,-1.0,0.081319,0.0
1833,replace,9.7,231,-1.0,0.619048,0.0
1834,replace,1.3,231,-1.0,0.445887,0.0
1835,replace,-6.9,537,0.0,0.255121,0.0


In [178]:
#Select data and label
X = train_df.drop(columns = ['target','operation'])
y = train_df['target']


In [179]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from scipy.stats import spearmanr
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 11)

In [180]:
#Fit the model and predict

model.fit(X_train, y_train)
predictions1 = model.predict(X_test)


In [181]:
#Print performance of the model
print('Mean Absolute Error =', mean_absolute_error(y_test, predictions1))
print('Mean Absolute Percentage Error = ', mean_absolute_percentage_error(y_test, predictions1))

Mean Absolute Error = 6.187791511647249
Mean Absolute Percentage Error =  166390228793968.66


In [182]:
rho, p = spearmanr(y_test, predictions1)
print('Spearman Correlation Coefficient =', rho.round(3))

Spearman Correlation Coefficient = -0.229



k_folds = 5
learning_rate = 1e-4
kfold = KFold(n_splits=k_folds, shuffle=True)
dataset = EnzymesDataset(df.reset_index(drop=True))
train_loss_history = []
test_loss_history = []
train_rho_history = []
test_rho_history = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    train_dl = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=32, sampler=train_subsampler)
    val_dl = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=32, sampler=test_subsampler)

    model = Conv1D_OneChannel()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    # defining the loss function
    criterion = nn.MSELoss()
    # checking if GPU is available
    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()
    
    for epoch in range(1, num_epochs + 1):
        train_loss , rho_train = train_epoch( model, optimizer, criterion, train_dl, epoch)
       

        
    
    
    test_loss , rho_test = test_epoch(model, criterion, val_dl)
        

    train_loss_history.append(train_loss)
    train_rho_history.append(rho_train)
    test_loss_history.append(test_loss)
    test_rho_history.append(rho_test)

    
    
    print(f'for fold {fold} : \n train_loss :  {train_loss}     test_loss : {test_loss} \n \n')
    
    
    
 


In [184]:
submission = model.predict(test_df)

In [187]:
test_df = pd.read_csv(path+ 'test.csv',index_col='seq_id')
test_df['tm']=submission

In [190]:
test_df

,protein_sequence,pH,data_source,tm
seq_id,,,,
31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,0.339834
31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,0.358451
31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,-0.560317
31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,0.339834
31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,0.339834
...,...,...,...,...
33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,0.405762
33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,0.405762
33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,0.500427


In [191]:

test_df = test_df.drop(columns=['protein_sequence','pH', 'data_source'])

In [192]:
test_df

,tm
seq_id,
31390,0.339834
31391,0.358451
31392,-0.560317
31393,0.339834
31394,0.339834
...,...
33798,0.405762
33799,0.405762
33800,0.500427


In [193]:
test_df.to_csv('Effect_mutation_2nd_method.csv', index=True)